# Deep Learning for NLP with PyTorch

저자: [Robert Guthrie](//github.com/rguthrie3/DeepLearningForNLPInPytorch)  
원본: http://pytorch.org/tutorials/beginner/deep_learning_nlp_tutorial.html  
역자: [Don Kim](//github.com/dgkim5360)

이 튜토리얼을 통해서 Pytorch를 이용한 딥러닝의 핵심을 전달하려 한다. 여기서 소개되는 많은 개념들 (computation graph abstraction, autograd 등)은 Pytorch에만 특별히 있는 것이 아니고, 여타 딥러닝 도구에서도 찾아 볼 수 있을 것이다.

특히 이 튜토리얼은 딥러닝 framework (Tensorflow, Theano, Keras, Dynet)을 전혀 사용하지 않은 사람들을 대상으로 NLP에 국한해서 진행한다. 또한 NLP의 기본 지식(part-of-speech tagging, language modeling 등)과 기초 AI 수업 (예를 들어 Russell과 Norvig의 교재)에서 배우는 정도의 neural networks 기본 지식은 알고 있다고 가정한다. 기초 AI 수업에서는 feed-forward neural network에 대한 기본적인 backpropagation을 소개하고, network는 linearity와 non-linearity의 조합의 연속으로 이루어 진다는 점을 알려줄 것이다. 이 튜토리얼은 이러한 기초 지식을 바탕으로 딥러닝 코드를 직접 짜보는 것을 목적으로 한다.

이 글은 _모델_ 에 관한 것이지, 데이터에 대한 것이 아니다. 여기서 다루는 모델의 weight들이 training 과정에 따라 어떻게 변화하는지 잘 볼 수 있도록 낮은 차원에서의 test 예제를 소개할 뿐이다. 실제 데이터를 가지고 해보고 싶다면 이 튜토리얼의 어떤 모델이든지 가져다가 사용해보길 바란다.

## Introduction to PyTorch
원본: http://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html

### Introduction to Torch's Tensor Library

딥러닝 계산은 모두 tensor를 통해 이루어진다. Tensor는 matrix를 일반화한 개념으로, matrix가 2방향으로 index를 가질 수 있듯이 tensor는 2개 이상의 index를 가질 수 있다. 머지않아 tensor가 정확히 어떤 의미를 가지는 지 알아볼 것이다.

우선 우리가 tensor를 가지고 뭘 할 수 있는 지 한 번 보자.

In [1]:
# Author: Robert Guthrie
# Translator: Don Kim
import torch
torch.manual_seed(1)

#### Creating Tensors
Tensor는 `torch.Tensor()` 함수가 Python list를 받아서 생성된다.

In [2]:
# 주어진 데이터를 가지고 torch.Tensor를 만들어 보자.
# 1차원 tensor인 vector를 만들어 보겠다.
V_data = [1., 2., 3.]
V = torch.Tensor(V_data)
print(V)

# 이번엔 2차원 tensor인 matrix다.
M_data = [[1., 2., 3.], [4., 5., 6.]]
M = torch.Tensor(M_data)
print(M)

# 2x2x2 크기의 3D tensor를 만들어 보자.
T_data = [[[1., 2.], [3., 4.]],
          [[5., 6.], [7., 8.]]]
T = torch.Tensor(T_data)
print(T)


 1
 2
 3
[torch.FloatTensor of size 3]


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



그래서 3D tensor가 뭔데? 이렇게 생각해 보자.

Vector의 원소 하나 하나는 숫자값(scalar)이다.  
Matrix의 원소 하나 하나는 벡터다.  
그렇다면 3D tensor의 원소 하나 하나로는 matrix가 나오는 것이다!

용어 정리: 앞으로 이 튜토리얼에서의 "tensor"는 `torch.Tensor` object를 의미하는 용어로 사용할 것이다. Vector와 matrix도 `torch.Tensor`의 차원이 1, 2인 간단한 경우로 속한다. 차원을 명시할 필요가 있을 경우에는 꼭 밝힐 것이다. 예를 들어 3차원 tensor를 가지고 설명할 경우에는 그냥 "tensor"가 아닌 "3D tensor"라고 쓸 것이다.

In [3]:
# V에 index로 접근하면 scalar를 받는다.
print(V[0])

# M에 index로 접근하면 vector를 받는다.
print(M[0])

# T에 index로 접근하면 matrix를 받는다.
print(T[0])

1.0

 1
 2
 3
[torch.FloatTensor of size 3]


 1  2
 3  4
[torch.FloatTensor of size 2x2]



여러 자료 유형에 대해서 텐서를 만들 수 있다. 이미 위 코드에서 봤듯이, 기본 세팅은 `Float`이다. Integer 형태의 tensor를 만들 때는 `torch.LongTensor()`를 사용하자. 그 외 다른 경우를 위해서는 documentation을 체크해보길 바란다. 하지만 대부분의 경우에 `Float`과 `Long`로 충분할 것이다.

`torch.randn()`을 통해서 원하는 차원의 난수 tensor를 생성할 수도 있다.

In [4]:
x = torch.randn((3, 4, 5))
print(x)


(0 ,.,.) = 
  0.6614  0.2669  0.0617  0.6213 -0.4519
 -0.1661 -1.5228  0.3817 -1.0276 -0.5631
 -0.8923 -0.0583 -0.1955 -0.9656  0.4224
  0.2673 -0.4212 -0.5107 -1.5727 -0.1232

(1 ,.,.) = 
  3.5870 -1.8313  1.5987 -1.2770  0.3255
 -0.4791  1.3790  2.5286  0.4107 -0.9880
 -0.9081  0.5423  0.1103 -2.2590  0.6067
 -0.1383  0.8310 -0.2477 -0.8029  0.2366

(2 ,.,.) = 
  0.2857  0.6898 -0.6331  0.8795 -0.6842
  0.4533  0.2912 -0.8317 -0.5525  0.6355
 -0.3968 -0.6571 -1.6428  0.9803 -0.0421
 -0.8206  0.3133 -1.1352  0.3773 -0.2824
[torch.FloatTensor of size 3x4x5]



#### Operations with Tensors
Tensor간의 operation은 아마도 우리가 익숙한 방식대로 작동할 것이다.

In [5]:
x = torch.Tensor([1., 2., 3.])
y = torch.Tensor([4., 5., 6.])
z = x + y
print(z)


 5
 7
 9
[torch.FloatTensor of size 3]



[Documentation](//pytorch.org/docs/torch.html)을 보면 우리가 tensor를 통해 할 수 있는 수 많은 operation 리스트를 볼 수 있을 것이다. 그 중에는 수학적인 연산의 개념을 넘어서는 operation도 있다.

그 중에서 자주 사용하게 될 concatenation을 사용해 보자. `torch.cat`은 나중에 이 튜토리얼에서도 사용하게 될 것이다.

In [6]:
# torch.concat은 기본값으로, 첫 번째 차원을 기준으로 붙인다. (행별로 붙여라)
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 = torch.cat((x_1, y_1))
print(z_1)

# 열별로 붙이려면
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
# 두 번째 argument에 붙이길 원하는 차원을 선택해주면 된다.
z_2 = torch.cat((x_2, y_2), 1)
print(z_2)

# 붙이고자 하는 tensor들이 서로 맞지 않으면, torch는 불만을 표시한다.
torch.cat((x_1, x_2))


-2.5667 -1.4303  0.5009  0.5438 -0.4057
 1.1341 -1.1115  0.3501 -0.7703 -0.1473
 0.6272  1.0935  0.0939  1.2381 -1.3459
 0.5119 -0.6933 -0.1668 -0.9999 -1.6476
 0.8098  0.0554  1.1340 -0.5326  0.6592
[torch.FloatTensor of size 5x5]


-1.5964 -0.3769 -3.1020 -0.0020 -1.0952  0.6016  0.6984 -0.8005
-0.0995 -0.7213  1.2708  1.5381  1.4673  1.5951 -1.5279  1.0156
[torch.FloatTensor of size 2x8]



RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 5 and 3 in dimension 1 at /pytorch/torch/lib/TH/generic/THTensorMath.c:2897

#### Reshaping Tensors
Tensor의 모양을 바꿔주는 `.view()` method는 대단히 많이 이용되는데, 그 이유로는 많은 neural network의 부품들이 자기의 input이 어떤 특정 모양이길 바라고 있기 때문이다. 당신의 데이터를 그 부품에게 주기 전에 모양을 바꿔줄 필요가 종종 있을 것이다.

In [7]:
x = torch.randn(2, 3, 4)
print(x)
print(x.view(2, 12))  # 2x12 tensor로 바꿔준다.
# 한 차원을 -1로 표시한다면, 해당 차원의 크기는 자동 계산될 수 있다.
# 따라서 아래 코드는 위 코드와 같은 결과를 준다.
print(x.view(2, -1))


(0 ,.,.) = 
 -0.2020 -1.2865  0.8231 -0.6101
 -1.2960 -0.9434  0.6684  1.1628
 -0.3229  1.8782 -0.5666  0.4016

(1 ,.,.) = 
 -0.1153  0.3170  0.5629  0.8662
 -0.3528  0.3482  1.1371 -0.3339
 -1.4724  0.7296 -0.1312 -0.6368
[torch.FloatTensor of size 2x3x4]



Columns 0 to 9 
-0.2020 -1.2865  0.8231 -0.6101 -1.2960 -0.9434  0.6684  1.1628 -0.3229  1.8782
-0.1153  0.3170  0.5629  0.8662 -0.3528  0.3482  1.1371 -0.3339 -1.4724  0.7296

Columns 10 to 11 
-0.5666  0.4016
-0.1312 -0.6368
[torch.FloatTensor of size 2x12]



Columns 0 to 9 
-0.2020 -1.2865  0.8231 -0.6101 -1.2960 -0.9434  0.6684  1.1628 -0.3229  1.8782
-0.1153  0.3170  0.5629  0.8662 -0.3528  0.3482  1.1371 -0.3339 -1.4724  0.7296

Columns 10 to 11 
-0.5666  0.4016
-0.1312 -0.6368
[torch.FloatTensor of size 2x12]



### Computational Graphs and Automatic Differentiation

Computation graph는 효율적인 딥러닝 개발을 위해 필수적인 개념이다. 왜냐하면 computation graph가 우리 대신에 back propagation gradient를 계산해주기 때문이다. Computation graph를 간단하게 말하자면, 어떻게 데이터가 결합돼서 output으로 계산이 된 것인지를 담은 기록장이다. 어느 parameter가 어느 연산과 연계되었는지를 모두 기록하기 때문에 computation graph는 미분을 계산할 수 있을 정도로 충분한 정보를 갖게 된다. 지금까지 설명한 것이 확실하게 와닿지 않을 수 있으므로, Pytorch의 핵심 class 중 하나인 `autograd.Variable`가 어떻게 작동하는 지 직접 보려고 한다.

먼저 프로그래머의 관점에서 생각해보겠다. 위에서 우리가 만든 `torch.Tensor` object에는 무엇이 담겨져 있을까? 당연하게도 데이터가 있을 것이고, 그 모양(shape) 정보도 있을 것이고, 기타 등등이 있을 것이다. 근데 두 tensor가 더해질 때를 생각해 보자. 그 결과로 받는 tensor도 그 데이터와 모양 정도는 알고 있겠지만, 그 tensor 입장에서 본인이 다른 두 tensor의 합이라는 사실을 알고 있을 리가 없다 (마찬가지로 tensor는 자기가 파일에서 읽어서 생성된 tensor인지, 다른 operation으로 생성된 것인지 알 수 없다).

`Variable` class는 자기 자신이 어떤 작업이 결과로 만들어진 것인지를 계속 기록한다. 코드를 보자.

In [8]:
# Variable은 tensor object를 받아서 시작한다.
x = torch.autograd.Variable(torch.Tensor([1., 2., 3.]),
                            requires_grad=True)
# .data를 통해서 기존 tensor 데이터를 확인할 수 있다.
print(x.data)

# Tensor 간에 할 수 있던 연산 모두 Variable에서 가능하다.
y = torch.autograd.Variable(torch.Tensor([4., 5., 6.]),
                            requires_grad=True)
z = x + y
print(z.data)

# 그리고 z는 뭔가를 더 알고 있다.
print(z.grad_fn)


 1
 2
 3
[torch.FloatTensor of size 3]


 5
 7
 9
[torch.FloatTensor of size 3]



`Variable`은 무엇이 자신을 만들었는지 안다. `z`는 자기 자신이 파일에서 읽어들여서 만들어진 것도 아니고, 곱셈 연산이나 지수 연산 등의 결과로 만들어진 게 아님을 아는 것이다.
`z.grad_fn`을 따라가보면 우리는 `x`와 `y`를 찾을 수 있게 된다.

하지만 그래서 어떻게 gradient를 계산할 수 있다는 걸까?

In [9]:
# z의 모든 원소를 더하겠다.
s = z.sum()
print(s)
print(s.grad_fn)

Variable containing:
 21
[torch.FloatTensor of size 1]



이제 `z`의 총합 `s`를 `x`의 첫 번째 변수에 대한 미분값이 뭘까? 수식으로 간단하게 쓰자면 우리가 구하고 싶은 것은 이거다.

\begin{equation}
\frac{\partial s}{\partial x_0}
\end{equation}

`s`는 자기가 tensor `z`의 합으로 만들어졌다는 것을 안다. `z`는 `x + y`의 결과라는 것을 안다. 따라서

\begin{equation}
s = \overbrace{x_0 + y_0}^{z_0} + \overbrace{x_1 + y_1}^{z_1} + \overbrace{x_2 + y_2}^{z_2}
\end{equation}

이고 `s`는 우리가 원하는 미분값이 1이라고 말해줄 수 있을만 한 충분한 정보를 다 갖고 있는 셈이다!

물론 지금까지 설명한 것으로는 진짜로 미분을 어떻게 계산하는 지를 완벽하게 설명할 순 없다. 요점은 `s`가 미분을 계산하기 위한 재료를 계속 가지고 다닌다는 것이다. 실제로 Pytorch 개발자들은 `sum()`과 `+` 연산 자체가 스스로 gradient를 계산하는 방법을 알고, back propagation 알고리즘을 수행할 수 있도록 고안했다. 더 자세한 내용은 이 튜토리얼의 범위를 벗어나므로 여기까지만 설명하겠다.

이제 직접 Pytorch로 gradient를 계산하고 위에서 설명한 것과 맞는지 확인해보자. 참고로, 만약 아래 cell을 여러 번 실행한다면 gradient는 누적 합산된다. 이것은 Pytorch가 의도한 설계로, gradient를 `.grad`로 _누적_ 시키는 것이 여러 모델에서 편리하기 때문이다.

In [10]:
# .backward()을 통해 variable `s`는 자신으로부터 시작되는 backprop을 계산한다.
s.backward()
print(x.grad)

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]



좋은 딥러닝 프로그래머가 되려면 아래 cell에서 어떤 일이 일어나는 지를 꼭 잘 이해해야 한다.

In [11]:
x = torch.randn((2, 2))
y = torch.randn((2, 2))
z = x + y  # 얘네들은 Tensor이므로, backprop은 할 수 없다.

var_x = torch.autograd.Variable(x, requires_grad=True)
var_y = torch.autograd.Variable(y, requires_grad=True)
# 우리는 이미 var_z가 gradient를 계산하기 위해 충분한 재료를 갖고 있음을 알고 있다.
var_z = var_x + var_y
print(var_z.grad_fn)

var_z_data = var_z.data  # var_z의 Tensor object를 빼오자...
# 그리고 새로운 variable로 태운다.
new_var_z = torch.autograd.Variable(var_z_data)

# ... 그러면 new_var_z는 x, y까지 backprop할 수 있는 정보가 남아있을까?
# NO!
print(new_var_z.grad_fn)
# 왜 이럴까? 우리는 var_z에서 tensor를 빼내서 (var_z.data를 통해) 복사했다.
# 이 tensor는 자기가 어떻게 계산된 것인지 전혀 모른다.
# 그리고 그 tensor를 new_var_z에 태웠고, new_var_z는 tensor가 아는 만큼 알게 된다.
# 그러니까 var_z_data가 모른다면, new_var_z 역시 알 길이 없다.
# 결국 우리는 variable이 가진 자기 역사를 끊어버린 것이다.

None


`autograd.Variable`의 연산에 관해서 기본적이지만 몹시 중요한 규칙 하나를 얘기하겠다. 그리고 이것은 Pytorch에 국한된 것이 아니고 모든 주요 딥러닝 도구를 사용함에 있어서 적용되는 개념이다.

__Loss function으로부터 network 구성 요소에까지 backpropagation을 통해 error를 계산하기 원한다면, 그 과정에서의 `Variable` chain을 절대! 끊으면 안된다. 만약 그 chain을 끊는다면 loss는 network의 구성 요소가 존재하는지 조차 모를 것이고, parameter들은 업데이트될 수가 없다.__

볼드체로 진지하게 쓴 이유는, 이로 인한 문제가 당신의 network를 굉장히 미묘하게 괴롭힐 수 있기 때문이다 (나중에 직접 보여주겠다). 또한 이런 문제는 코드 레벨에서 에러가 나거나 불평하지 않으므로, 훨씬 조심스럽게 접근해야 한다.

## Deep Learning with PyTorch
원본: http://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html

### Deep Learning Building Blocks: Affine Maps, Non-linearities and Objectives
딥러닝은 linearity와 non-linearity를 똑똑하게 결합하는 방법으로 구성된다. Non-linearity가 개입함으로써 model은 강력해진다. 여기서는 Pytorch의 핵심 모듈들을 가지고 놀면서 objective function도 구성해보고, 모델이 어떻게 학습하는지 보겠다.

#### Affine Maps
딥러닝을 이끄는 핵심 구성 요소 중 하나는 affine map으로, 다음 함수 $f$로 표시할 수 있다.

\begin{equation}
f(x) = Ax + b
\end{equation}

여기서 $A$는 matrix, $b$는 vector로, 우리가 구하고자 하는 parameter가 된다. $b$는 _bias_라는 이름으로 불리기도 한다.

Pytorch 및 여타 딥러닝 framework들 모두 전통적인 선형 대수의 방법과는 조금 다르게 행렬 연산을 하는데, input의 열(column) 대신 행(row)를 기준으로 연산한다. 이는 output의 $i$ 번째 행을 계산하기 위해 input의 $i$ 번째 행을 mapping $A$로 보내고, bias를 더한다는 말이다. 아래 예제를 보자.

In [12]:
# R^5에서 R^3로 보내는 map, parameter A, b를 갖는다.
lin = torch.nn.Linear(5, 3)
# data는 2x5이고, lin은 5에서 3으로 보내는 map이다... lin읉 통해 mapping이 될까?
data = torch.autograd.Variable(torch.randn(2, 5))
print(lin(data))  # Yes

Variable containing:
 0.4491 -0.1994  0.0853
-0.0392  0.2063  0.0476
[torch.FloatTensor of size 2x3]



#### Non-linearities
Non-linearity가 대접받아야 하는 이유를 알기 위해 우선적으로 짚고 넘어가야할 사실이 있다. 2개의 affine map이 있다고 해보자.

\begin{align}
f(x) &= Ax + b \\
g(x) &= Cx + d
\end{align}

그러면 $f(g(x))$는 무엇이 될까?

\begin{equation}
f(g(x)) = A(Cx+d) + b = ACx + (Ad+b)
\end{equation}

$AC$는 matrix고 $Ad+b$는 vector다. 그러니까 결국 그 결과는 다시 affine map이 된다.

따라서, neural network가 그저 affine map들의 긴 chain이라면 그 network는 단 하나의 affine map과 같은 레벨의 모델이 될 뿐임을 알 수 있다.

여기서 affine layer 사이에 non-linearity를 끼워 넣는다면, 하나의 affine map과 다른 훨씬 강력한 모델을 구축할 수 있다.

핵심적인 non-linearity가 여럿 있는데, 그 중에서도 $\text{tanh}(x)$, $\sigma(x)$, $\text{ReLU}(x)$가 널리 쓰인다. 누군가 궁금할 수 있다, "왜 하필 쟤네들이지? 저거 말고 다른 non-linear function은 너무 많잖아". 그 이유는 학습에 필수적인 gradient를 쉽게 계산할 수 있기 때문이다. 예를 들어 sigmoid의 경우를 들겠다.

\begin{equation}
\frac{d\sigma}{dx} = \sigma(x)\left(1-\sigma(x)\right)
\end{equation}

잠깐 첨언하자면, AI 관련 수업에서 $\sigma$를 기본 non-linearity로 채택해서 neural network를 설명했을 수도 있지만 실제로 sigmoid는 주로 사용되지 않는다. 그 이유는 argument $x$의 크기(absolute value)가 커질 수록 굉장히 빠르게 0으로 사라지기 때문이다. 작은 gradient는 곧 학습시키기 어려워짐을 의미한다. 그래서 대부분 $\text{tanh}$나 $\text{ReLU}$를 기본 non-linearity로 사용한다.

In [13]:
# Pytorch에서 대부분의 non-linearity는 nn.functional에 있다.
# Affine map과 다르게 non-linearity는 통상 parameter가 없음을 알아두자.
# 즉, 학습 과정에서 업데이트할 weight이 없다는 말이다.
data = torch.autograd.Variable(torch.randn(2, 2))
print(data)
print(torch.nn.functional.relu(data))

Variable containing:
-0.1277 -0.9501
 1.4064 -0.4549
[torch.FloatTensor of size 2x2]

Variable containing:
 0.0000  0.0000
 1.4064  0.0000
[torch.FloatTensor of size 2x2]



#### Softmax and Similarities
$\text{Softmax}(x)$ 함수 역시 non-linearity 중 하나인데, network의 마지막 operation으로 사용되기 때문에 특별하다. 마지막으로 사용되는 이유는 softmax가 실수 vector를 받아서 확률 분포로 뱉어내기 때문이다. Softmax의 정의를 해보겠다. $x$를 실수 vector라고 하자 (양수이든 음수이든 상관없고 어떠한 제한도 없다). 그럼 $\text{Softmax}(x)$의 $i$ 번째 값은 다음과 같다.

\begin{equation}
\frac{\exp (x_i)}{\sum_j \exp (x_j)}
\end{equation}

이 결과는 틀림없이 확률 분포가 된다: 모든 값들이 non-negative하고, 그 합이 1이 된다.

Softmax는 input의 값들을 각각 지수 함수로 태워서 non-negative하게 만든 뒤, 합이 1이 되도록 (normalizing constant로) 나눠준 거라고도 생각할 수 있다.

In [14]:
# Softmax 역시 nn.functional에 있다.
data = torch.autograd.Variable(torch.randn(5))
print(data)
print(torch.nn.functional.softmax(data, dim=0))
# 확률 분포이기에 합이 1이다!
print(torch.nn.functional.softmax(data, dim=0).sum())
# log_softmax도 있다.
print(torch.nn.functional.log_softmax(data, dim=0))

Variable containing:
 0.5838
-0.9699
-0.9469
-0.2330
 0.6424
[torch.FloatTensor of size 5]

Variable containing:
 0.3413
 0.0722
 0.0739
 0.1508
 0.3619
[torch.FloatTensor of size 5]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
-1.0750
-2.6287
-2.6057
-1.8918
-1.0164
[torch.FloatTensor of size 5]



#### Objective Functions
Objective function은 network가 학습하기 위한 목표로 최소화하고자 하는 함수를 말한다 (_loss function_ 또는 _cost function_이라고도 불린다). Objective function은 우선 training 자료를 하나 골라서 neural network에 넣어서 돌린 후에 output의 loss를 계산한다. 모델의 parameter들은 loss function의 미분을 계산해서 업데이트하게 된다. 직관적으로 생각해봤을 때, 모델의 답이 맞다고 확신하고 있는데 그 답이 틀렸다면 loss는 높을 것이고, 그 답이 맞다면 loss는 낮을 것이다.

Training example에 대해서 loss function을 최소화하는 것에 깔려있는 생각은, network가 부디 일반적으로 잘 맞출 수 있는 모델로 발전하길 바라고, dev set/test set/production에서 나오는 새로운 데이터가 들어왔을 때 작은 loss가 나오길 바란다는 것이다.

Loss function의 예로 _negative log likelihood loss_를 들 수 있는데, 이것은 multi-class 분류 문제에서 굉장히 널리 쓰이는 objective이다. Supervised multi-class 분류 문제에서 negative log likelihood loss를 사용하면 올바른 output에 대한 negative log probability를 최소화하는 방향 (반대로 말하면 올바른 output일 경우의 log 확률을 최대화하는 방향)으로 network을 훈련시킬 수 있게 된다.

### Optimization and Training
그래서 loss function으로 우리는 뭘 계산할 수 있는가? 이미 우리는 `autograd.Variable`이 우리가 적용한 연산을 기억해서 gradient를 계산할 수 있는 정보를 기록해두고 있다고 알고 있다. 우리가 사용할 loss 역시 `autograd.Variable`이어서 loss를 계산하는데 필요한 모든 parameter에 대한 gradient를 계산할 수 있다! 그러고 나면 일반적인 gradient 업데이트를 할 수 있게 된다. $\theta$를 parameter라고 하고, $L(\theta)$를 loss function, $\eta$를 양수의 learning rate라고 표시하면, 다음과 같이 parameter 업데이트를 할 수 있다.

\begin{equation}
\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta)
\end{equation}

위와 같은 "vanilla" gradient 업데이트보다 일을 잘 할 수 있는 수 많은 알고리즘들이 있고, 계속 연구가 진행되고 있다. 많은 사람들이 train 단계에서 learning rate를 변화시켜보곤 한다. 이런 이론에 관심이 없다면 딱히 어떤 알고리즘이 어떻게 작동하는지 신경쓰지 않아도 된다. Torch는 `torch.optim` 패키지를 통해 많은 방법을 완전 투명하게 개발해두었다. 세상에서 제일 간단한 gradient update를 사용하는 것이나 그것보다 복잡한 알고리즘을 사용하는 것이나 Pytorch에서는 똑같은 방법으로 사용할 수 있다. 여러 update 알고리즘과 그 알고리즘에 사용될 여러 parameter (예를 들어 여러 가지 초기 learning rate)들을 시험해보는 것은 network 성능을 올리는데 중요한 일이다. 종종 기본 SGD를 Adam이나 RMSProp으로만 바꿔줘도 성능이 크게 올라가는 것을 볼 때가 있다.

### Creating Network Components in Pytorch
NLP로 들어가기 전에, Pytorch에서 affine map과 non-linearity만을 이용해서 network를 구성해보는 연습을 해보자. 또한 Pytorch에 내장된 negative log likelihood를 통해서 loss function을 어떻게 계산하고 어떻게 backpropagation을 해서 parameter를 업데이트하는지도 보게 될 것이다.

모든 network 구성 요소들은 `nn.Module`을 상속해야하고 `.forward()` method를 덮어씌워 놓아야 한다. Boilerplate과 관련돼서는 이 두 가지만 지키면 된다.

`nn.Module`을 상속하는 구성 요소들은 많은 기능을 탑재하게 된다. 예를 들어, 학습시키고자 하는 parameter들을 추적하면서, CPU와 GPU 사이를 오가도록 `.cpu()`와 `.cuda()`를 사용해서 바꿀 수도 있는 것이다.

Sparse bag-of-words 표현을 받아서 "English"인지 "Spanish"인지에 대한 확률을 내뱉는 network를 예제로 작성해보겠다. 이번 모델은 단순한 logistic regression이다.

#### Example: Logistic Regression Bag-of-Words Classifier
우리의 모델은 sparse한 BoW represenation을 label에 따른 log probability로 변환(map)해 줄 것이다. 우리 단어장에 모든 단어들을 index로 지정해두겠다. 예를 들어 단어장에 "hello"와 "world" 두 단어만이 index 0과 1로 등록되어 있다면, "hello hello hello hello"를 변환한 BoW vector는 $[4, 0]$이 되고, "hello world hello world"는 $[2, 2]$가 될 것이다. 일반적으로는

\begin{equation}
\left[\text{Count}(\text{hello}), \text{Count}(\text{world})\right]
\end{equation}

가 될 것이다.

BoW vector를 $x$라고 지칭하겠다. 이제 우리의 network가 내보내는 output은

\begin{equation}
\log \text{Softmax}(Ax+b)
\end{equation}

으로, 다시 말하면 input을 affine map에 태운 뒤 log softmax를 하는 것이다.

In [15]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix는 각 단어를 단어장의 중복되지 않는 index로 연결해서
# BoW vector로 변환할 수 있도록 해준다.
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [16]:
class BoWClassifier(torch.nn.Module):  # nn.Module을 상속받는다.
    def __init__(self, num_labels, vocab_size):
        # nn.Module의 __init__을 호출해라.
        # 그냥 무조건 해라 (nn.Module을 상속하는 경우)
        super(BoWClassifier, self).__init__()
        
        # 우리가 필요로 하는 parameter들을 정의한다.
        # 이번 예제에서는 affine mapping의 parameter인 A와 b가 필요하다.
        # Torch가 탑재한 nn.Linear()를 통해 affine map을 사용할 수 있다.
        # 왜 input 차원이 vocab_size이고 output 차원이 num_labels인지
        # 확실히 알고 있어야 한다!
        self.linear = torch.nn.Linear(vocab_size, num_labels)
        
        # 잠깐! non-linearity인 log softmax는 parameter가 필요 없다!
        # 따라서 여기서는 신경쓸 필요가 없다.

    def forward(self, bow_vec):
        # Input을 linear layer로 보내고, 다시 log_softmax로 보낸다.
        # 많은 non-linearity 및 다른 함수들이 nn.functional에 탑재되어 있다.
        return torch.nn.functional.log_softmax(self.linear(bow_vec), dim=1)


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [17]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# model은 자신의 parameter를 알고 있다. 첫 output은 A, 두 번째는 b를 출력할 것이다.
# Pytorch 개발자들의 Python 마법으로 Module class의 __init__ 함수에서
# component를 할당할 때, 그러니까 이 예제에서는
# self.linear = nn.Linear(...)
# 할 때, 이를 감지해서 당신의 module이 nn.Linear의 parameter에 관해 알도록 되어있다.
for param in model.parameters():
    print(param)

# model을 돌리려면, BoW vector를 넣어야 한다.
# 근데 autograd.Variable로 포장해서 보내야 한다.
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(torch.autograd.Variable(bow_vector))
print(log_probs)

Parameter containing:

Columns 0 to 9 
-0.0372 -0.0723  0.0818 -0.0668 -0.1102  0.0445 -0.1418 -0.0419  0.1002  0.0733
-0.1794 -0.1151  0.1199  0.0430 -0.0692 -0.0741  0.1253  0.1414  0.1889  0.0572

Columns 10 to 19 
 0.1670 -0.1338  0.0017 -0.0579 -0.1097 -0.1103  0.0494  0.0205 -0.1001 -0.1382
 0.0947 -0.0158 -0.1147 -0.1921  0.1195 -0.0285  0.0806  0.0097 -0.1830 -0.0962

Columns 20 to 25 
-0.1625  0.1922 -0.0334 -0.0453 -0.0721 -0.0981
 0.0552 -0.0397 -0.1894  0.0688  0.0075 -0.0630
[torch.FloatTensor of size 2x26]

Parameter containing:
1.00000e-02 *
  7.9225
  4.1404
[torch.FloatTensor of size 2]

Variable containing:
-0.6198 -0.7723
[torch.FloatTensor of size 1x2]



여기서 어떤 값들이 `ENGLISH`와 `SPANISH`에 연결된 log probability인지 우리가 아직 지정한 적이 없다. 따라서 학습을 시키려면 output label을 지정해야 한다.

In [18]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

자, 이제 학습시켜 보자! 그 과정으로 우리는 training 데이터를 집어 넣어서 log probability를 받고, loss function을 계산한 뒤, loss function의 gradient를 구하고, gradient step으로 parameter를 업데이트하면 된다.

Loss function은 Torch에서 `nn` 패키지로 제공한다. `nn.NLLLoss()`가 우리가 원하는 negative log likelihood loss다. 또한 `torch.optim` 패키지에서 최적화를 위한 기능을 이용할 수 있는데, 여기는 간단한 `SGD`를 사용하겠다.

`NLLLoss`의 _input_ 으로 log probability를 가진 vector와, target label을 필요로 한다는 것을 알아두자. 우리를 위해서 log probability까지 계산해주지는 않는 것이다. 이것이 우리가 위에서 log softmax를 마지막 layer로 추가한 이유이다. 반면에 `nn.CrossEntropyLoss()`는 log softmax를 대신 해주는 `NLLLoss()`와 같다.

In [19]:
# 비교를 위해, 학습이 시작되기 전에 test 데이터를 돌려본다.
for instance, label in test_data:
    bow_vec = torch.autograd.Variable(make_bow_vector(instance,
                                                      word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# "creo"에 해당하는 parameter column을 출력해본다.
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# 보통 우리는 training 데이터를 여러 번 반복해서 계산한다.
# 100번은 실제 데이터를 사용할 경우에는 너무 많은 반복이지만,
# 지금 예제에서는 training 데이터의 개수가 두 개이므로 적당할 것이다.
# 통상적으로 5와 30 사이의 반복 횟수 (epoch)가 적당하다.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Pytorch는 gradient를 누적시킨다는 점을 기억하자.
        # 따라서 training 데이터가 들어갈 때마다 기존 gradient를 지워야 한다.
        model.zero_grad()
        
        # Step 2. BoW vector를 만들고 그것을 정수 타입의 Variable에 넣는다.
        # 예를 들어 target이 SPANISH라면 우리는 Variable에 정수 0을 넣을 것이다.
        # 그러고 나면 loss function은 log probability의 0번째값은 SPANISH 라벨에
        # 해당하는 log probability임을 알 수 있다.
        bow_vec = torch.autograd.Variable(make_bow_vector(instance,
                                                          word_to_ix))
        target = torch.autograd.Variable(make_target(label, label_to_ix))
        
        # Step 3. forward pass
        log_probs = model(bow_vec)
        
        # Step 4. loss, gradients를 계산한 뒤, optimizer.step()를 통해
        # parameter를 업데이트한다.
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

for instance, label in test_data:
    bow_vec = torch.autograd.Variable(make_bow_vector(instance,
                                                      word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Spanish index에 맞는 parameter는 높아야 하고,
# English index에 맞는 parameter는 낮아야 한다!
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
-0.7991 -0.5974
[torch.FloatTensor of size 1x2]

Variable containing:
-0.6153 -0.7776
[torch.FloatTensor of size 1x2]

Variable containing:
 0.1670
 0.0947
[torch.FloatTensor of size 2]

Variable containing:
-0.1582 -1.9219
[torch.FloatTensor of size 1x2]

Variable containing:
-2.4182 -0.0933
[torch.FloatTensor of size 1x2]

Variable containing:
 0.6040
-0.3423
[torch.FloatTensor of size 2]

